In [26]:
import pandas as pd

aave_holders_raw = pd.read_csv('aave-token-holders.csv')
geist_holders_raw = pd.read_csv('geist-token-holders.csv')
radiant_holders_raw = pd.read_csv('radiant-token-holders.csv')

# Remove rows with the zero address
aave_holders_first_clean = aave_holders_raw[aave_holders_raw['HolderAddress'] != '0x0000000000000000000000000000000000000000']
geist_holders_first_clean = geist_holders_raw[geist_holders_raw['HolderAddress'] != '0x0000000000000000000000000000000000000000']
radiant_holders_first_clean = radiant_holders_raw[radiant_holders_raw['HolderAddress'] != '0x0000000000000000000000000000000000000000']

# Remove rows with zero balance
aave_holders_second_clean = aave_holders_first_clean[aave_holders_first_clean['Balance'] != 0]
geist_holders_second_clean = geist_holders_first_clean[geist_holders_first_clean['Balance'] != 0]
radiant_holders_second_clean = radiant_holders_first_clean[radiant_holders_first_clean['Balance'] != 0]

